In [11]:
import os
import csv
import numpy
import pandas
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import skfuzzy

In [16]:

data = {
    'current': [],
    'rawIntensity': [],
    'temperature': [],
    'voltage': [],
    'pwm': []
}

with open('dataset.csv', mode='r') as file:
    reader = csv.DictReader(file)

    for row in reader:
        data['current'].append(float(row['current']))
        data['rawIntensity'].append(int(row['rawIntensity']))
        data['temperature'].append(int(float(row['temperature'])))
        data['voltage'].append(float(row['voltage']))
        data['pwm'].append(int(row['pwm']))

df = pandas.DataFrame(data)

In [17]:
df.head()

,current,rawIntensity,temperature,voltage,pwm
0,37.5,1011,25,0.860,150
1,35.8,1004,25,0.900,100
2,37.7,1016,25,0.892,100
3,36.8,1010,25,0.892,100
4,37.1,1010,25,0.896,100


In [22]:
X = df[['current', 'rawIntensity', 'temperature', 'voltage']].values
y = df['pwm'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:


def gaussian_mf(x, mean, sigma):
    return numpy.exp(-((x - mean) ** 2) / (2 * sigma ** 2))

def anfis_inference(inputs, mf_params, rule_weights):
    mf_values = []
    for i, input_value in enumerate(inputs):
        param = list(mf_params.values())[i]
        mf_values.append(gaussian_mf(input_value, param[0], param[1]))

    weighted_sum = 0
    weight_total = 0
    for i, weight in enumerate(rule_weights):
        weighted_sum += mf_values[i] * weight
        weight_total += weight

    return weighted_sum / weight_total if weight_total != 0 else 0



In [27]:
mf_params = {
    'current': [0.6, 0.1],  # [mean, sigma]
    'rawIntensity': [100, 10],
    'temperature': [30, 5],
    'voltage': [1.5, 0.2]
}

In [25]:
rule_weights = [1.0, 0.6, 0.6, 0.6]

predictions = []
for x in X_test:
    predictions.append(anfis_inference(x, mf_params, rule_weights))

mse = numpy.mean((numpy.array(predictions) - y_test) ** 2)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 13505.1845904679


In [20]:
print(x)

[3.720e+01 1.011e+03 2.500e+01 8.520e-01]


In [21]:
print(len(X_train))

2684
